In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS'

course_list = ['청도','포항','중문']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
}

folder_type = ['rgb','ndvi','thumb']

In [4]:
target_folders = glob.glob(os.path.join(out_folder,'**/*'))
target_folders

['D:\\ToAWS\\MGC001\\20230811', 'D:\\ToAWS\\MGC002\\20230823']

In [5]:
target_folder = target_folders[0]

In [6]:
course_id = course_ids.get('포항')
target_date = os.path.split(target_folder)[-1]
course_id, target_date

('MGC001', '20230811')

In [7]:
dataJson_ = glob.glob(os.path.join(out_folder,course_id,target_date,'**/photo*.json'), recursive=True)[0]
dataJson_ 

'D:\\ToAWS\\MGC001\\20230811\\photo.json'

In [8]:
with open(dataJson_, "r") as _json:
  dataJson= json.load(_json)

In [9]:
len(dataJson)

7571

In [10]:
dataJson[0]

{'id': '20230811114038_1293386534167_362205185833_131509000',
 'originalFileJPG': 'original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해 4H\\2023_0811_포항CC_동해 4H_1.5m\\20230811_1140-1147_물리적피해,과습피해,플레이어답압,토양병,페어리링\\DJI_20230811114038_0001_D.JPG',
 'originalFileGrp': ['D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해 4H\\2023_0811_포항CC_동해 4H_1.5m\\20230811_1140-1147_물리적피해,과습피해,플레이어답압,토양병,페어리링\\DJI_20230811114038_0001_D.JPG',
  'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해 4H\\2023_0811_포항CC_동해 4H_1.5m\\20230811_1140-1147_물리적피해,과습피해,플레이어답압,토양병,페어리링\\DJI_20230811114038_0001_MS_G.TIF',
  'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해 4H\\2023_0811_포항CC_동해 4H_1.5m\\20230811_1140-1147_물리적피해,과습피해,플레이어답압,토양병,페어리링\\DJI_20230811114038_0001_MS_NIR.TIF',
  'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해 4H\\2023_0811_포항CC_동해 4H_1.5m\\20230811_1140-1147_물리적피해,과습피해,플레이어답압,토양병,페어리링\\DJI_20230811114038_0001_MS_R.TIF',
  'D:\\original\\2023_0811_포항CC_비온다음날_데이터 1차 분류_완\\동해 4H\\2023_0811_포항CC_동

In [11]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [22]:
for index_, row_ in enumerate(dataJson):
  newrow = {}
  newrow ['id'] = row_['id']
  newrow ['metadata'] = course_id+target_date+row_['info']['area']+row_['info']['desc']
  newrow ['date'] = target_date
  newrow ['area'] = row_['info']['area']
  newrow ['desc'] = row_['info']['desc']
  newrow ['json'] = json.loads(json.dumps(row_), parse_float=Decimal)
  print(index_, end='\r')
  table.put_item(
    Item= newrow
  )

In [14]:
target_date

'20230811'

In [30]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError



response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [31]:
len(data)

121